In [ ]:
pip install tensorflow==2.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 41.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninst

## Prerequisite

In [ ]:
pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
pip install sklearn-evaluation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.4 MB/s eta 0:00:00


In [ ]:
import numpy as np

# Data utilities
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import os.path
import torch
import h5py

# Machine learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer, f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import cross_val_score

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/haloAdd.csv

--2024-06-17 01:18:38--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/haloAdd.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1362849 (1.3M) [text/plain]
Saving to: ‘haloAdd.csv’

haloAdd.csv         100%[===================>]   1.30M  6.12MB/s    in 0.2s    

2024-06-17 01:18:38 (6.12 MB/s) - ‘haloAdd.csv’ saved [1362849/1362849]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Utility Function

In [ ]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [ ]:
# Utility function: Display model score(Loss & Accuracy) across all sets.

def display_model_score(model, train, val, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

  val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
  print('Val loss: ', val_score[0])
  print('Val accuracy: ', val_score[1])
  print('-'*70)

  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

In [ ]:
def equal_error_rate(y_true, y_pred):
    n_imp = tf.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

In [ ]:
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

In [ ]:
def error_rate(testing_labels, predicted_testing_labels):
    from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, classification_report, recall_score, confusion_matrix
    import numpy as np

    bootstrap_performances = list()
    performances = list()
    f1_performances = list()
    sn = list()
    sp = list()
    Y = np.array(testing_labels)  # convert list of groundtruths to numpy
    Yhat = np.array(predicted_testing_labels)  # same same for predictions
    n_samples = len(Y)  # get number of samples
    n_bootstrap = 1000  # number of bootstrap iterations

    for i in range(n_bootstrap):  # for each bootstrap draw
        subset = np.random.choice(n_samples, n_samples, replace=True)
        # create a random subset of your predictions/targets with replacement
        Y_subset = Y[subset]
        Yhat_subset = Yhat[subset]

        bootstrap_performances.append(matthews_corrcoef(Y_subset, Yhat_subset))
        performances.append(accuracy_score(Y_subset, Yhat_subset))
        f1_performances.append(f1_score(Y_subset, Yhat_subset))
        sn.append(recall_score(Y_subset, Yhat_subset))

        tn, fp, fn, tp = confusion_matrix(Y_subset, Yhat_subset).ravel()
        sp.append(tn / (tn + fp))

    sd_mcc = np.std(bootstrap_performances)  # compute std deviation over the bootstrapped performances
    sd_acc = np.std(performances)
    sd_f1 = np.std(f1_performances)
    sd_sn = np.std(sn)
    sd_sp = np.std(sp)

    print('acc:', accuracy_score(testing_labels, predicted_testing_labels))
    print('f1:', f1_score(testing_labels, predicted_testing_labels))
    print('mcc:', matthews_corrcoef(testing_labels, predicted_testing_labels))

    tn, fp, fn, tp = confusion_matrix(testing_labels, predicted_testing_labels).ravel()
    specificity = tn / (tn + fp)

    print('sn:', recall_score(testing_labels, predicted_testing_labels))
    print('sp:', specificity)
    print('sd_acc:', sd_acc)
    print('sd_f1:', sd_f1)
    print('sd_mcc:', sd_mcc)
    print('sd_sn:', sd_sn)
    print('sd_sp:', sd_sp)
    print(classification_report(testing_labels, predicted_testing_labels))

    return (sd_acc, sd_mcc, sd_f1)


In [ ]:
def conf_matrix(confusion_matrix_data):
  from mlxtend.plotting import plot_confusion_matrix
  fig, ax = plot_confusion_matrix(conf_mat =confusion_matrix_data,
                                show_absolute=True,
                                show_normed=True,
                                #display_labels=class_dict.values(),
                                colorbar=True)
  labels = ['Non-halophilic', 'Halophilic']
  ax.set_xticklabels([''] + labels)
  ax.set_yticklabels([''] + labels)
  plt.show()


In [ ]:
def mcc(clf,X,y):
  y_pred = clf.predict(X)
  mcc = matthews_corrcoef(y, y_pred)
  return mcc

In [ ]:
def matthews_correlation_coefficient(y_true, y_pred):
    from keras import backend as K
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

In [ ]:
def std_acc(clf,X,y):
  from sklearn.metrics import accuracy_score
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  y_pred = clf.predict(X)
  Y = np.array(y) # convert list of groundtruths to numpy
  Yhat = np.array(y_pred) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append( accuracy_score(y[subset], y_pred[subset]) )
  sd_acc = np.std(bootstrap_performances)*1.96
  return sd_acc

In [ ]:
def std_f1(clf,X,y):
  from sklearn.metrics import f1_score
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  y_pred = clf.predict(X)
  Y = np.array(y) # convert list of groundtruths to numpy
  Yhat = np.array(y_pred) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append(f1_score(y[subset], y_pred[subset]) )
  sd_f1 = np.std(bootstrap_performances)*1.96
  return sd_f1

In [ ]:
def std_mcc(clf,X,y):
  from sklearn.metrics import matthews_corrcoef
  import numpy as np

  bootstrap_performances = list()
  performances = list()
  y_pred = clf.predict(X)
  Y = np.array(y) # convert list of groundtruths to numpy
  Yhat = np.array(y_pred) # same same for predictions
  n_samples = len(Y) # get number of samples
  n_bootstrap = 1000 # number of bootstrap iterations
  for i in range(n_bootstrap): # for each bootstrap draw
    subset = np.random.choice(n_samples, n_samples, replace=True)
    # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
    bootstrap_performances.append(matthews_corrcoef(y[subset], y_pred[subset]) )
  sd_mcc = np.std(bootstrap_performances)*1.96
  return sd_mcc

In [ ]:
def sp():
  TP = confusion_matrix_data[1,1]
  TN = confusion_matrix_data[0,0]
  FP = confusion_matrix_data[0,1]
  FN = confusion_matrix_data[1,0]

  print(TP,TN, FP, FN)

  sn = TP / float(TP + FN)
  print(sn)
  sp = TN / float(TN + FP)
  print(sp)

## Open embedding file

In [ ]:
path = '/content/drive/MyDrive/Halophilic/ESM2_3B_HaloAdd' #/content/drive/MyDrive/Halophilic/ESM2_650M' #'/content/drive/MyDrive/Halophilic/ESM2_3B/' #'/content/drive/MyDrive/HalophilicESM2'
suffix = '.pt'
delete = []
embeddings = []

In [ ]:
proteins = []

In [ ]:
#!rsync -av /content/drive/MyDrive/Halophilic/ESM2_3B_HaloAdd/ /content/drive/MyDrive/Multiclass/

In [ ]:
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo80.csv

--2024-06-17 01:20:38--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo80.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 933288 (911K) [text/plain]
Saving to: ‘Halo80.csv’

Halo80.csv          100%[===================>] 911.41K  4.77MB/s    in 0.2s    

2024-06-17 01:20:39 (4.77 MB/s) - ‘Halo80.csv’ saved [933288/933288]



In [ ]:
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo60.csv

--2024-06-17 01:20:40--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo60.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 794406 (776K) [text/plain]
Saving to: ‘Halo60.csv’

Halo60.csv          100%[===================>] 775.79K  4.20MB/s    in 0.2s    

2024-06-17 01:20:40 (4.20 MB/s) - ‘Halo60.csv’ saved [794406/794406]



In [ ]:
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo40.csv

--2024-06-17 01:20:40--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo40.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 686703 (671K) [text/plain]
Saving to: ‘Halo40.csv’

Halo40.csv          100%[===================>] 670.61K  3.72MB/s    in 0.2s    

2024-06-17 01:20:41 (3.72 MB/s) - ‘Halo40.csv’ saved [686703/686703]



In [ ]:
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo20.csv

--2024-06-17 01:20:41--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo20.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 601036 (587K) [text/plain]
Saving to: ‘Halo20.csv’

Halo20.csv          100%[===================>] 586.95K  3.57MB/s    in 0.2s    

2024-06-17 01:20:42 (3.57 MB/s) - ‘Halo20.csv’ saved [601036/601036]



In [ ]:
# Basic Protocol 3 — Step 5
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/haloAdd.csv
#!wget https://raw.githubusercontent.com/33220311/halophilic/main/haloNath.csv


--2024-06-17 01:20:42--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/haloAdd.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1362849 (1.3M) [text/plain]
Saving to: ‘haloAdd.csv.1’

haloAdd.csv.1       100%[===================>]   1.30M  6.07MB/s    in 0.2s    

2024-06-17 01:20:42 (6.07 MB/s) - ‘haloAdd.csv.1’ saved [1362849/1362849]



In [ ]:
!wget https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo-Hidrofob.csv

--2024-06-17 01:20:42--  https://raw.githubusercontent.com/33220311/halophilic/main/dataset/Halo-Hidrofob.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46407 (45K) [text/plain]
Saving to: ‘Halo-Hidrofob.csv’

Halo-Hidrofob.csv   100%[===================>]  45.32K  --.-KB/s    in 0.04s   

2024-06-17 01:20:43 (1.03 MB/s) - ‘Halo-Hidrofob.csv’ saved [46407/46407]



In [ ]:
annotations = read_csv('haloAdd.csv')#('haloAdd.csv')#('uniqueAnnotations1024.csv')
#annotations = read_csv('annotationsUnique.csv')

In [ ]:
annotations[:3]

,identifier,sequence,label,set
0,QSG09462,MRFFDRLAERIDAVDSVVSVGLDPDPDRLPESVADADLPRFQFNRR...,1,train
1,QSG11440,MTRVIHTGDTHLGYQQYHEPARREDFLSAFRQVIEDAVAEDVDAVV...,1,train
2,BCB06597,MLRVAITERPQWRELAHQLGFHFHTIEGEPYWTEDAYYQFTLTQIE...,1,train


In [ ]:
for filename in annotations.identifier:
    file = os.path.join(path, filename + suffix)
    if (os.path.exists(file)):
      result = torch.load(file)
      rep = result.get('mean_representations')
      val = rep.values()
      val = list(val)
      val = np.array(val[0])
      val = np.reshape(val,(-1,1280))
      proteins.append((filename,val))
    else:
      delete.append(filename)

In [ ]:
#proteins

In [ ]:
# Basic Protocol 3 — Step 6
train_set = annotations[annotations.set == "train"]
test_set = annotations[annotations.set == "test"]

In [ ]:
print(f"The train set contains {len(train_set)} samples, and we will test on {len(test_set)} samples.")

The train set contains 5670 samples, and we will test on 1356 samples.


In [ ]:
# Basic Protocol 3 — Step 7

training_identifiers = train_set.identifier.values
training_labels = train_set.label.values
print(len(training_identifiers))
print(len(training_labels))

5670
5670


In [ ]:
testing_identifiers = test_set.identifier.values
testing_labels = test_set.label.values
print(len(testing_identifiers))
print(len(testing_labels))

1356
1356


In [ ]:
seq = dict(proteins)
delete = list()

In [ ]:
training_embeddings = list()
for identifier in training_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            training_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [ ]:
testing_embeddings = list()
for identifier in testing_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            testing_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [ ]:
np.where(training_identifiers=='Mes206')

(array([], dtype=int64),)

In [ ]:
#training_identifiers = np.delete(training_identifiers,4728)
#training_labels = np.delete(training_labels,4728)

In [ ]:
len(training_labels)

5670

In [ ]:
len(training_embeddings)

5670

In [ ]:
1024# A sanity check: make sure that the numbers are equal!
assert(len(training_identifiers) == len(training_embeddings))
assert(len(testing_identifiers) == len(testing_embeddings))

In [ ]:
len(training_embeddings)

5670

In [ ]:
delete

[]

## Training

In [ ]:
arr_train = np.array(training_embeddings)
nsample, nx, ny = arr_train.shape
train_dataset = arr_train.reshape((nsample, nx*ny))
train_dataset.shape

(5670, 2560)

In [ ]:
arr_test = np.array(testing_embeddings)
nsample, nx, ny = arr_test.shape
test_dataset = arr_test.reshape((nsample, nx*ny))
test_dataset.shape

(1356, 2560)

### LR

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
import plotly.express as px

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
lr = LogisticRegression()
lr_history = lr.fit(train_dataset, training_labels)
lr.score(test_dataset,testing_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.911504424778761

In [ ]:
#grid_scorer = {'accuracy':make_scorer(accuracy_score),'f1':make_scorer(f1_score),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}
grid_scorer = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score),
    'mcc': make_scorer(matthews_corrcoef, greater_is_better=True),
    'sensitivity': make_scorer(recall_score, greater_is_better=True),
    'specificity': make_scorer(specificity_score, greater_is_better=True)
}

In [ ]:
scores = cross_validate(lr, train_dataset, training_labels, scoring= grid_scorer, cv=10)
scores

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{'fit_time': array([1.87491012, 1.63103771, 1.47561836, 1.58759141, 1.50165629,
        1.50874567, 1.45145917, 1.50145268, 1.73859954, 2.130126  ]),
 'score_time': array([0.02675605, 0.01243687, 0.00918317, 0.01271272, 0.01479506,
        0.0170505 , 0.00919056, 0.01037455, 0.01911163, 0.02816081]),
 'test_accuracy': array([0.8377425 , 0.85890653, 0.95943563, 0.93650794, 0.93650794,
        0.96825397, 0.89417989, 0.90123457, 0.8712522 , 0.92239859]),
 'test_f1': array([0.87894737, 0.89924433, 0.96953642, 0.952     , 0.952     ,
        0.97650131, 0.925     , 0.93120393, 0.90604891, 0.94527363]),
 'test_mcc': array([0.63359123, 0.66805479, 0.91052971, 0.86111345, 0.86111345,
        0.92771248, 0.75229534, 0.7720879 , 0.70190718, 0.82255421]),
 'test_sensitivity': array([0.86753247, 0.92727273, 0.95064935, 0.92727273, 0.92727273,
        0.97142857, 0.96103896, 0.98441558, 0.91428571, 0.98958333]),
 'test_specificity': array([0.77472527, 0.71428571, 0.97802198, 0.95604396, 0.95604396

source https://www.kaggle.com/code/jnikhilsai/cross-validation-with-linear-regression

In [ ]:
predicted_training_labels = lr.predict(train_dataset)
error_rate(training_labels, predicted_training_labels)

acc: 0.9589065255731922
f1: 0.9699703570047686
mcc: 0.9051979647211783
sn: 0.9776565341647181
sp: 0.9192751235584844
sd_acc: 0.002527210705383344
sd_f1: 0.001876249617416053
sd_mcc: 0.005847063383575142
sd_sn: 0.0022490624154456263
sd_sp: 0.006158646957328016
              precision    recall  f1-score   support

           0       0.95      0.92      0.93      1821
           1       0.96      0.98      0.97      3849

    accuracy                           0.96      5670
   macro avg       0.96      0.95      0.95      5670
weighted avg       0.96      0.96      0.96      5670



(0.002527210705383344, 0.005847063383575142, 0.001876249617416053)

In [ ]:
predicted_lr = lr.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_lr)
error_rate(testing_labels, predicted_lr)

acc: 0.911504424778761
f1: 0.9480968858131488
mcc: 0.6502439800240276
sn: 0.9375534644995723
sp: 0.7486631016042781
sd_acc: 0.007312496353402619
sd_f1: 0.004462908891728077
sd_mcc: 0.02907261244637941
sd_sn: 0.007047897749271824
sd_sp: 0.03163483302845615
              precision    recall  f1-score   support

           0       0.66      0.75      0.70       187
           1       0.96      0.94      0.95      1169

    accuracy                           0.91      1356
   macro avg       0.81      0.84      0.82      1356
weighted avg       0.92      0.91      0.91      1356



(0.007312496353402619, 0.02907261244637941, 0.004462908891728077)

In [ ]:
df =pd.DataFrame(train_dataset)
df.to_excel('/content/lr_train_dataset.xlsx')
df =pd.DataFrame(training_labels)
df.to_excel('/content/lr_training_labels.xlsx')
df =pd.DataFrame(predicted_training_labels)
df.to_excel('/content/lr_pred_train_labels.xlsx')

df =pd.DataFrame(test_dataset)
df.to_excel('/content/lr_test_dataset.xlsx')
df =pd.DataFrame(testing_labels)
df.to_excel('/content/lr_testing_labels.xlsx')
df =pd.DataFrame(predicted_lr)
df.to_excel('/content/lr_pred_test_labels.xlsx')

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_lr, labels=classes)
conf_matrix(confusion_matrix_data)


In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
#lr.save_weights('/result/LR.h5')
print(lr.coef_.shape)
lr_weights = lr.coef_
print(lr_weights)
print(np.max(lr_weights))
print(np.min(lr_weights))

### MLP with Grid Search

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
# Basic Protocol 3 — Step 8

multilayerperceptron = MLPClassifier(solver='lbfgs', random_state=10, max_iter=1000)

parameters = {
    'hidden_layer_sizes': [(2056,1024),(2056,),(1024,)],
    #'learning_rate_init': [0.001, 0.0001, 0.01],
    #'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['adam'],
}

In [ ]:
grid_scorer ={'accuracy':make_scorer(accuracy_score),'f1':make_scorer(f1_score),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
# Basic Protocol 3 — Step 9

classifiers = GridSearchCV(multilayerperceptron, parameters, cv=10, scoring=grid_scorer, refit='mcc')
history = classifiers.fit(train_dataset, training_labels)
classifier = classifiers.best_estimator_

In [ ]:
params = classifier.get_params()
params

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (2056, 1024),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 1000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 10,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [ ]:
#from keras.models import model_from_json

# Save the best model's architecture to a JSON file
#model_json = grid_result.best_estimator_.model.to_json()
#with open("/content/drive/MyDrive/Halophilic/MLPESM_best_model.json", "w") as json_file:
#    json_file.write(model_json)

# Save the best model's weights to an HDF5 file
#grid_result.best_estimator_.model.save_weights("/content/drive/MyDrive/Halophilic/MLPESM_best_model_weights.h5")

NameError: ignored

In [ ]:
import pandas as pd
from pandas import DataFrame

In [ ]:
DataFrame(classifiers.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_solver,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,split3_test_mcc,split4_test_mcc,split5_test_mcc,split6_test_mcc,split7_test_mcc,split8_test_mcc,split9_test_mcc,mean_test_mcc,std_test_mcc,rank_test_mcc
0,184.813537,25.161848,0.053326,0.010312,"(2056, 1024)",adam,"{'hidden_layer_sizes': (2056, 1024), 'solver':...",0.827160,0.855379,0.971781,...,0.896197,0.870773,0.929476,0.824579,0.808279,0.752410,0.804864,0.808134,0.105952,1
1,163.099791,12.781107,0.043167,0.011069,"(2056,)",adam,"{'hidden_layer_sizes': (2056,), 'solver': 'adam'}",0.816578,0.853616,0.954145,...,0.872989,0.911235,0.917829,0.805631,0.811196,0.758739,0.765492,0.797705,0.107002,2
2,135.653790,20.399292,0.083915,0.044345,"(1024,)",adam,"{'hidden_layer_sizes': (1024,), 'solver': 'adam'}",0.811287,0.850088,0.943563,...,0.876365,0.890067,0.928483,0.803340,0.791941,0.756348,0.792661,0.793652,0.107022,3


In [ ]:
df =pd.DataFrame(classifiers.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Halophilic/ESM23BMLP20.xlsx')


In [ ]:
predicted_training_labels = classifier.predict(train_dataset)
error_rate(training_labels, predicted_training_labels)

acc: 0.9998236331569665
f1: 0.9998700792516564
mcc: 0.9995956619714399
sn: [1.0, 0.9997362869198312, 1.0, 0.9997400571874188, 0.9997384937238494, 0.9994814622763806, 0.9997386987196237, 0.9997391077484998, 0.9994753410283316, 0.9992262058292494, 1.0, 0.9992201715622563, 0.999741468459152, 1.0, 1.0, 0.9992130115424974, 0.9997398543184183, 1.0, 0.9997419354838709, 0.9994821336095288, 0.9994814622763806, 0.999739989599584, 1.0, 1.0, 1.0, 1.0, 0.9992221934145709, 1.0, 1.0, 0.9994697773064687, 0.9994850669412977, 0.9997354497354497, 1.0, 1.0, 0.9997390396659708, 0.9994769874476988, 0.9997419354838709, 1.0, 1.0, 0.9997446373850868, 1.0, 0.9994756161510225, 0.9997360781208762, 0.9992084432717678, 1.0, 0.9997379454926625, 0.9997396511325176, 0.9997380141472361, 1.0, 0.9997407311381903, 0.9997392438070404, 0.9994869163673679, 1.0, 0.9997393117831074, 1.0, 1.0, 0.999213630406291, 1.0, 1.0, 0.9997403946002077, 0.9997437211686314, 1.0, 1.0, 0.9997424008243173, 0.9997387669801463, 1.0, 1.0, 0.99948

(0.00017235074686758426, 0.0003952747433627781, 0.00012694378021800782)

In [ ]:
# Basic Protocol 3 — Step 10

predicted_mlp = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_mlp)
error_rate(testing_labels, predicted_mlp)

acc: 0.9092920353982301
f1: 0.9467762873215058
mcc: 0.6423311893792609
sn: [0.9341216216216216, 0.9283276450511946, 0.9436379163108455, 0.9370689655172414, 0.9428571428571428, 0.9459227467811159, 0.9308600337268128, 0.9327659574468085, 0.9458762886597938, 0.946551724137931, 0.9376601195559351, 0.9429051217464316, 0.9440619621342513, 0.9459915611814346, 0.9317988064791134, 0.9299743808710503, 0.9416809605488851, 0.9438687392055267, 0.9328743545611016, 0.9388458225667528, 0.9457167090754877, 0.924114671163575, 0.9384088964927289, 0.9421140939597316, 0.9349871685201027, 0.9351535836177475, 0.9417721518987342, 0.9382924767540152, 0.9383033419023136, 0.9378723404255319, 0.9316823228010248, 0.9319148936170213, 0.9343986543313709, 0.941025641025641, 0.9298986486486487, 0.9385281385281385, 0.93071000855432, 0.9317988064791134, 0.9394205443371378, 0.937984496124031, 0.9448685326547922, 0.9346517626827171, 0.9333902647309992, 0.9444444444444444, 0.9397071490094746, 0.9378238341968912, 0.92746113

(0.007717964245438891, 0.030770667245480488, 0.0046979580186572245)

In [ ]:
df =pd.DataFrame(train_dataset)
df.to_excel('/content/mlp_train_dataset.xlsx')
df =pd.DataFrame(training_labels)
df.to_excel('/content/mlp_training_labels.xlsx')
df =pd.DataFrame(predicted_training_labels)
df.to_excel('/content/mlp_pred_train_labels.xlsx')

df =pd.DataFrame(test_dataset)
df.to_excel('/content/mlp_test_dataset.xlsx')
df =pd.DataFrame(testing_labels)
df.to_excel('/content/mlp_testing_labels.xlsx')
df =pd.DataFrame(predicted_mlp)
df.to_excel('/content/mlp_pred_test_labels.xlsx')

In [ ]:
from pandas import DataFrame
cv_results = DataFrame(classifiers.cv_results_)
print(cv_results)

In [ ]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_mlp, labels=classes)
conf_matrix(confusion_matrix_data)

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
#classifier.save_weights('/result/MLP.h5')

#fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = classifier.coefs_[0].min(), classifier.coefs_[0].max()
#for coef, ax in zip(classifier.coefs_[0].T, axes.ravel()):
#    ax.matshow(coef.reshape(1024, ), cmap=plt.cm.gray, vmin=0.5 * vmin, vmax=0.5 * vmax)
#    ax.set_xticks(())
#    ax.set_yticks(())

#plt.show()

In [ ]:
vmax

In [ ]:
weights = classifier.coefs_
print(len(weights))
print("shape",weights[0].shape,"content",weights[0])
print("shape",weights[1].shape,"content",weights[1])
#print("shape",weights[2].shape,"content",weights[2])


### CNN with GridSearch

In [ ]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D, AveragePooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from time import time

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
n_timesteps, n_features, n_outputs =train_dataset.shape[0], train_dataset.shape[1], Y_train.shape[1]

In [ ]:
n_epochs = 30 # 30
n_epochs_cv = 10 # 10  # reduce number of epochs for cross validation for performance reason

n_cv = 10
validation_ratio = 0.10

In [ ]:
def create_cnn_model(pool_type='max', conv_activation='tanh', dropout_rate=0.1, kernel=3, optimizer='adam'):
    # create model
    model = Sequential()

    # first layer: convolution
    #model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv1D(filters=128, kernel_size=kernel, activation=conv_activation,input_shape=(n_features,1)))
    # second series of layers: convolution, pooling, and dropout
    model.add(Conv1D(32, kernel_size=kernel, activation=conv_activation))
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    # third series of layers: convolution, pooling, and dropout
    model.add(Conv1D(64, kernel_size=kernel, activation=conv_activation))   # 32   10
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    model.add(Conv1D(64, kernel_size=kernel, activation=conv_activation))   # 32   10
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    # fourth series
    model.add(Flatten())
    model.add(Dense(64, activation='sigmoid')) # 64
    # add a dropout layer if rate is not null
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        optimizer= optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'],
        )
    return model

In [ ]:
cnn = create_cnn_model()

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2558, 128)         512       
                                                                 
 conv1d_1 (Conv1D)           (None, 2556, 32)          12320     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1278, 32)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1278, 32)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1276, 64)          6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 638, 64)          0         
 1D)                                                    

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=150, verbose=1)

In [ ]:
# fit network
history = cnn.fit(train_dataset, Y_train,
                        validation_data=(test_dataset, Y_test),
                        callbacks=[es],
                        epochs=10, verbose=1
                        )

Epoch 1/10
178/178 [==============================] - 41s 209ms/step - loss: 0.2990 - accuracy: 0.8644 - recall: 0.9348 - precision: 0.8741 - true_positives: 3598.0000 - true_negatives: 1303.0000 - false_positives: 518.0000 - false_negatives: 251.0000 - val_loss: 0.2537 - val_accuracy: 0.8791 - val_recall: 0.8973 - val_precision: 0.9597 - val_true_positives: 1049.0000 - val_true_negatives: 143.0000 - val_false_positives: 44.0000 - val_false_negatives: 120.0000
Epoch 2/10
178/178 [==============================] - 52s 290ms/step - loss: 0.1929 - accuracy: 0.9206 - recall: 0.9499 - precision: 0.9343 - true_positives: 3656.0000 - true_negatives: 1564.0000 - false_positives: 257.0000 - false_negatives: 193.0000 - val_loss: 0.2439 - val_accuracy: 0.8923 - val_recall: 0.9068 - val_precision: 0.9663 - val_true_positives: 1060.0000 - val_true_negatives: 150.0000 - val_false_positives: 37.0000 - val_false_negatives: 109.0000
Epoch 3/10
178/178 [==============================] - 40s 224ms/step -

In [ ]:
# optimize model
start = time()

# create model
model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
# define parameters and values for grid search
param_grid = {
    'pool_type': ['max','average'], #max
    'conv_activation': ['relu', 'tanh'],#relu
    'epochs': [10],
    'kernel':[1,2,3,4], #2
    'optimizer':['adam','sgd'] #adam
}

<ipython-input-61-d21e2dc07263>:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_cnn_model, verbose=1)


In [ ]:
grid_scorer = {'accuracy':make_scorer(accuracy_score),'f1':make_scorer(f1_score),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=10, scoring=grid_scorer, refit='mcc')
grid_result = grid.fit(train_dataset, Y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
classifier = grid.best_estimator_

params = classifier.get_params()
params

In [ ]:
from keras.models import model_from_json

# Save the best model's architecture to a JSON file
model_json = grid_result.best_estimator_.model.to_json()
with open("/content/drive/MyDrive/Halophilic/CNNESM_best_model.json", "w") as json_file:
    json_file.write(model_json)

# Save the best model's weights to an HDF5 file
grid_result.best_estimator_.model.save_weights("/content/drive/MyDrive/Halophilic/CNNESM_best_model_weights.h5")

In [ ]:
predicted_training_labels = classifier.predict(train_dataset)
error_rate(training_labels, predicted_training_labels)

In [ ]:
predicted_cnn = classifier.predict(test_dataset)
error_rate(testing_labels, predicted_cnn)

In [ ]:
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))

In [ ]:
# summarize results
#print('time for grid search = {:.0f} sec'.format(time()-start))
#display_cv_results(grid_result)

In [ ]:
df =pd.DataFrame(grid.cv_results_)
#new_path = '/content/test.xls'
#writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Halophilic/ESM23BCNN2.xlsx')

In [ ]:
df =pd.DataFrame(train_dataset)
df.to_excel('/content/cnn_train_dataset.xlsx')
df =pd.DataFrame(training_labels)
df.to_excel('/content/cnn_training_labels.xlsx')
df =pd.DataFrame(predicted_training_labels)
df.to_excel('/content/cnn_pred_train_labels.xlsx')

df =pd.DataFrame(test_dataset)
df.to_excel('/content/cnn_test_dataset.xlsx')
df =pd.DataFrame(testing_labels)
df.to_excel('/content/cnn_testing_labels.xlsx')
df =pd.DataFrame(predicted_cnn)
df.to_excel('/content/cnn_pred_test_labels.xlsx')

In [ ]:
#from pandas import DataFrame
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_conv_activation','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_cnn, labels=classes)
conf_matrix(confusion_matrix_data)
confusion_matrix_data

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [ ]:
sn = TP / float(TP + FN)
print(sn)

In [ ]:
sp = TN / float(TN + FP)
print(sp)

In [ ]:
confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

### CNN using Pytorch

In [ ]:
from torch import nn, optim

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
X_train, X_test= train_dataset[:5632], test_dataset[:1280]
y_train, y_test = training_labels[:5632], testing_labels[:1280]

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train).type(torch.FloatTensor) # data type is long

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor) # data type is long

batch_size = 128
num_epochs = 100

train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=1, stride=1)
        self.tanh1 = nn.Tanh()
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.tanh2 = nn.Tanh()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1)
        self.tanh3 = nn.Tanh()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.tanh4 = nn.Tanh()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(320, 128)
        self.sigmoid1 = nn.Sigmoid()
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid2 = nn.Sigmoid()


    def forward(self, x):
        out = self.conv1(x)
        out = self.tanh1(out)
        out = self.conv2(out)
        out = self.tanh2(out)
        out = self.maxpool1(out)
        out = self.conv3(out)
        out = self.tanh3(out)
        out = self.maxpool2(out)
        out = self.conv4(out)
        out = self.tanh4(out)
        out = self.maxpool3(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.sigmoid2(out)
        return out

# Create CNN
model = CNNModel()

# Cross Entropy Loss
error = nn.BCELoss()

# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# CNN model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
    for i, (embeddings, labels) in enumerate(train_loader):
        train = embeddings
        labels = labels.view(-1, 1)


        optimizer.zero_grad() # Clear gradients
        outputs = model(train) # Forward propagation
        loss = error(outputs, labels) # Calculate softmax and cross entropy loss
        loss.backward() # Calculating gradients
        optimizer.step() # Update parameters

        count += 1

        if count % 50 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0

            # Predict test dataset
            for embeddings, labels in test_loader:
                test = embeddings
                outputs = model(test) # Forward propagation
                predicted = torch.max(outputs.data, 1)[1] # Get predictions from the maximum value
                total += len(labels) # Total number of labels
                correct += (predicted == labels).sum() # Total correct predictions

            accuracy = 100.0 * correct.item() / total

            # store loss and iteration
            loss_list.append(loss.data.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 500 == 0:
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data.item(), accuracy))

Iteration: 500  Loss: 0.00020486551511567086  Accuracy: 8.671875 %
Iteration: 1000  Loss: 0.0003513967676553875  Accuracy: 8.671875 %
Iteration: 1500  Loss: 8.751754648983479e-05  Accuracy: 8.671875 %
Iteration: 2000  Loss: 8.642775355838239e-05  Accuracy: 8.671875 %
Iteration: 2500  Loss: 0.00010572530300123617  Accuracy: 8.671875 %
Iteration: 3000  Loss: 3.788506001001224e-05  Accuracy: 8.671875 %
Iteration: 3500  Loss: 2.9513119443436153e-05  Accuracy: 8.671875 %
Iteration: 4000  Loss: 5.871677058166824e-05  Accuracy: 8.671875 %


### LR with GridSearch

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [ ]:
lr = LogisticRegression()
lr.fit(train_dataset, training_labels)

LogisticRegression()

In [ ]:
hyper_params = [{'n_features_to_select': list(range(1, 1024))}]
rfe = RFE(lr)

In [ ]:
model_cv = GridSearchCV(estimator = rfe,
                        param_grid = hyper_params,
                        scoring= 'r2',
                        cv = 10,
                        verbose = 1,
                        return_train_score=True)

In [ ]:
model_cv.fit(train_dataset, training_labels)

Fitting 10 folds for each of 1023 candidates, totalling 10230 fits


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
df =pd.DataFrame(model_cv.cv_results_)
#new_path = '/content/test.xls'
#writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/SOM/LRESM1bMerged.xlsx')

In [ ]:
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
# final model
n_features_optimal = 10

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)
rfe = rfe.fit(X_train, y_train)

# predict prices of X_test
y_pred = lm.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

### Capsnet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras import initializers, models
from keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def CapsNet_Binary(input_shape, num_capsule, dim_capsule, routings=2):
  """
  Creates a CapsNet model architecture for binary classification.

  Args:
      input_shape: Shape of the input data (e.g., (None, 1024, 1)).
      num_capsule: Number of digit capsules (set to 1 for binary classification).
      dim_capsule: Dimension of each capsule vector.
      routings: Number of routing iterations between DigitCaps and PrimaryCaps.

  Returns:
      A Keras model representing the CapsNet architecture.
  """
  # Input layer
  inputs = layers.Input(shape=input_shape)

  # Convolution layer
  conv1 = layers.Conv1D(filters=256, kernel_size=9, activation='relu')(inputs)

  # Primary capsule layer
  primary_capsule = layers.Conv1D(filters=num_capsule * dim_capsule, kernel_size=1)(conv1)
  reshaped = layers.Reshape((num_capsule, dim_capsule))(primary_capsule)

  # Squashing function (modified for numerical stability)
  def squash(x):
    norm = tf.sqrt(tf.reduce_sum(tf.square(x), axis=-1, keepdims=True))
    squash_factor = (norm + 1) / (norm ** 2 + 1)
    return squash_factor * x

  squashed = layers.Lambda(squash)(reshaped)

  # Digit capsule layer
  digit_caps = squashed
  for _ in range(routings):
    digit_caps_hat = layers.Dense(num_capsule * dim_capsule, activation='relu')(digit_caps)
    digit_caps_hat = layers.Reshape((num_capsule, 1, dim_capsule))(digit_caps_hat)
    # Ensure both squashed and digit_caps_hat are passed as arguments
    agreement = layers.Lambda(lambda squashed, digit_caps_hat: tf.keras.backend.batch_dot(squashed, digit_caps_hat, axes=[1, 1]))([squashed, digit_caps_hat])
    agreement = layers.Activation('softmax')(agreement)
    digit_caps = layers.Lambda(lambda squashed, agreement: tf.keras.backend.batch_dot(agreement, squashed, axes=[1, 1]))([squashed, agreement])

  # Output layer (sigmoid activation for binary classification)
  outputs = layers.Dense(1, activation='sigmoid')(digit_caps)

  # Model definition
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model


# Example usage (replace with your actual input shape)
input_shape = (None, 1024, 1)
num_capsule = 1  # Set to 1 for binary classification
dim_capsule = 8
routings = 2

model = CapsNet_Binary(input_shape, num_capsule, dim_capsule, routings)

# Compile the model (binary cross-entropy loss for binary classification)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load your training data (X_train, y_train) - binary labels (0 or 1)
# ...

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save the model (optional)
model.save('capsnet_binary_model.h5')


TypeError: Exception encountered when calling layer "lambda_16" (type Lambda).

CapsNet_Binary.<locals>.<lambda>() missing 1 required positional argument: 'digit_caps_hat'

Call arguments received by layer "lambda_16" (type Lambda):
  • inputs=['tf.Tensor(shape=(None, 1, 8), dtype=float32)', 'tf.Tensor(shape=(None, 1, 1, 8), dtype=float32)']
  • mask=None
  • training=None

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def CapsNet(input_shape, num_capsule, dim_capsule, routings=3):
    """
    Creates a CapsNet model architecture.

    Args:
        input_shape: Shape of the input data (e.g., (1024, 1)).
        num_capsule: Number of digit capsules.
        dim_capsule: Dimension of each capsule vector.
        routings: Number of routing iterations between DigitCaps and PrimaryCaps.

    Returns:
        A Keras model representing the CapsNet architecture.
    """
    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Convolution layer
    conv1 = layers.Conv1D(filters=256, kernel_size=9, activation='relu')(inputs)

    # Primary capsule layer
    primary_capsule = layers.Conv1D(filters=num_capsule * dim_capsule, kernel_size=9, strides=2, padding='valid')(conv1)

    # Calculate the primary capsule output shape dynamically
    primary_capsule_output_shape = primary_capsule.shape
    reshaped = layers.Reshape((primary_capsule_output_shape[1] * primary_capsule_output_shape[2] // dim_capsule, dim_capsule))(primary_capsule)

    # Squashing function (modified for numerical stability)
    def squash(x):
        norm = tf.sqrt(tf.reduce_sum(tf.square(x), axis=-1, keepdims=True))
        squash_factor = (norm ** 2) / (1 + norm ** 2)
        return squash_factor * x / norm

    squashed = layers.Lambda(squash)(reshaped)

    # Digit capsule layer
    digit_caps = squashed
    for _ in range(routings):
        digit_caps_hat = layers.Dense(num_capsule * dim_capsule, activation='relu')(digit_caps)
        digit_caps_hat = layers.Reshape((num_capsule, dim_capsule))(digit_caps_hat)
        agreement = layers.Lambda(lambda caps: tf.keras.backend.batch_dot(caps, digit_caps, axes=[2, 1]))([squashed, digit_caps_hat])
        agreement = layers.Activation('softmax')(agreement)
        digit_caps = layers.Lambda(lambda x: tf.keras.backend.batch_dot(agreement, x, axes=[2, 1]))([digit_caps_hat, squashed])

    # Flatten the digit capsules output
    digit_caps_flat = layers.Flatten()(digit_caps)

    # Output layer
    outputs = layers.Dense(2, activation='softmax')(digit_caps_flat)

    # Model definition
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (1024, 1)  # Replace with your actual input shape
num_capsule = 32
dim_capsule = 8
routings = 3

model = CapsNet(input_shape, num_capsule, dim_capsule, routings)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load your training data (X_train, y_train)
# ...

# Example training data
import numpy as np
X_train = np.random.rand(100, 1024, 1)  # Dummy data, replace with actual data
Y_train = keras.utils.to_categorical(np.random.randint(2, size=(100, 1)), num_classes=2)

# Train the model
model.fit(X_train, Y_train, epochs=10)

# Save the model (optional)
model.save('capsnet_model.h5')


In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
error_rate(Y_train, predicted_training_labels)

In [ ]:
# Basic Protocol 3 — Step 10
predicted_testing_labels = model.predict(X_test)
predicted_testing_labels = np.where(predicted_testing_labels > 0.5, 1, 0)
error_rate(Y_test, predicted_testing_labels)

### RNN with GridSearch

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer, GRU
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras import layers

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
max_length = 1280

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
def create_rnn_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam'):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    rnn = SimpleRNN(units, activity_regularizer=l2(regularizer), return_sequences = True)(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(rnn)
    rnn2 =SimpleRNN(units, activity_regularizer=l2(dropout_rate))(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x2 = Dropout(dropout_rate)(rnn2)
    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x2)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),#solver,#
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
rnn_model = create_rnn_model()
rnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2560, 1)]         0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 2560, 50)          2600      
                                                                 
 dropout (Dropout)           (None, 2560, 50)          0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 7,701
Trainable params: 7,701
Non-trainable par

In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_rnn_model, verbose=1, epochs=5)
# define parameters and values for grid search

parameters = {
    'units':[50], # 1024
    #'learning_rate_init': [0.001, 0.01],
    'solver':['sgd', 'adam'],
    'epochs':[2,3,5,10]
    #'dropout_rate':[0.0,0.05, 0.1], #0.05
    #'regularizer':[0.0,0.05, 0.1],
}

<ipython-input-58-d4d62c73a518>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_rnn_model, verbose=1, epochs=5)


In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

Epoch 1/3
178/178 [==============================] - 283s 2s/step - loss: 0.4835 - accuracy: 0.7690
Epoch 2/3
178/178 [==============================] - 283s 2s/step - loss: 0.4084 - accuracy: 0.8162
Epoch 3/3
178/178 [==============================] - 281s 2s/step - loss: 0.3693 - accuracy: 0.8347


In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

{'verbose': 1,
 'epochs': 3,
 'solver': 'sgd',
 'units': 50,
 'build_fn': <function __main__.create_rnn_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam')>}

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_units','param_dropout_rate','param_regularizer','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Halophilic/ESM23BRNN2.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

178/178 [==============================] - 49s 276ms/step
acc: 0.8359788359788359
f1: 0.8831658291457287
mcc: 0.612743893963337
sn: [0.9175176747839748, 0.9179852320675106, 0.9206598586017282, 0.9136989862230309, 0.9228556485355649, 0.9105522426756546, 0.9111575646720669, 0.9089486042264545, 0.9121196222455404, 0.9102398761929327, 0.9131101456682852, 0.9048609305952691, 0.9146845915201655, 0.9174406604747162, 0.9151843258571797, 0.9089716684155299, 0.9177939646201873, 0.9113760041461518, 0.9055483870967742, 0.9124805800103574, 0.9134042001555613, 0.9092563702548102, 0.9202211690363349, 0.922875131164743, 0.9160747171796896, 0.9155776267642446, 0.9074410163339383, 0.9108860759493671, 0.9126775381378222, 0.9209968186638389, 0.9150360453141092, 0.917989417989418, 0.9191157347204161, 0.9124675324675324, 0.9086638830897703, 0.9181485355648535, 0.9130322580645162, 0.9137355584082156, 0.9129866736347008, 0.9159856996935649, 0.913545407901488, 0.916885159937074, 0.9173924518342571, 0.914511873

(0.004921827799755303, 0.011220447861780957, 0.003812693844120401)

In [ ]:
predicted_rnn = classifier.predict(X_test)
predicted_rnn = np.where(predicted_rnn > 0.5, 1, 0)
predicted_rnn = np.reshape(predicted_rnn,(len(predicted_rnn),)).astype(int)
error_rate(Y_test, predicted_rnn)

43/43 [==============================] - 17s 398ms/step
acc: 0.8296460176991151
f1: 0.9004739336492891
mcc: 0.31030129304726994
sn: [0.9028716216216216, 0.893344709897611, 0.8812980358667806, 0.8939655172413793, 0.8831932773109243, 0.9081545064377683, 0.8929173693086003, 0.8919148936170213, 0.9080756013745704, 0.8991379310344828, 0.9077711357813835, 0.887489504617968, 0.8907056798623064, 0.9122362869198313, 0.875532821824382, 0.8975234842015372, 0.8962264150943396, 0.9075993091537133, 0.9053356282271945, 0.9104220499569337, 0.9050042408821035, 0.8853288364249579, 0.8810949529512404, 0.8934563758389261, 0.88366124893071, 0.893344709897611, 0.8827004219409282, 0.8951817413355875, 0.8946015424164524, 0.8970212765957447, 0.89666951323655, 0.9012765957446809, 0.8999158957106812, 0.8940170940170941, 0.9054054054054054, 0.8909090909090909, 0.8956372968349017, 0.8951406649616368, 0.8762071992976295, 0.8854435831180018, 0.8888888888888888, 0.8856405846947549, 0.8855678906917165, 0.9017094017094

(0.010159137191586488, 0.03519226789090742, 0.006473745669038299)

### GRU with GridSearch

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer, GRU
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras import layers

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
max_length = 1280

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
def create_gru_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam'):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    gru = GRU(units, activity_regularizer=l2(regularizer), return_sequences = True)(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(gru)
    gru2 =GRU(units, activity_regularizer=l2(dropout_rate))(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x2 = Dropout(dropout_rate)(gru2)
    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x2)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),#solver,#
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
gru_model = create_gru_model()
gru_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2560, 1)]         0         
                                                                 
 gru (GRU)                   (None, 2560, 50)          7950      
                                                                 
 dropout_4 (Dropout)         (None, 2560, 50)          0         
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 23,301
Trainable params: 23,301
Non-trainable

In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_gru_model, verbose=1, batch_size = 256, epochs=10)
# define parameters and values for grid search

parameters = {
    'units':[50], # 1024
    #'learning_rate_init': [0.001, 0.01],
    'solver':['adam'],
    #'epochs':[3,5,10]
    #'dropout_rate':[0.0,0.05, 0.1], #0.05
    #'regularizer':[0.0,0.05, 0.1],
}

<ipython-input-77-c73ca1a6eab2>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_gru_model, verbose=1, batch_size = 256, epochs=10)


In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

Epoch 1/10
23/23 [==============================] - 143s 6s/step - loss: 0.6539 - accuracy: 0.6691
Epoch 2/10
23/23 [==============================] - 135s 6s/step - loss: 0.6276 - accuracy: 0.6788
Epoch 3/10
23/23 [==============================] - 137s 6s/step - loss: 0.6246 - accuracy: 0.6788
Epoch 4/10
23/23 [==============================] - 135s 6s/step - loss: 0.6228 - accuracy: 0.6788
Epoch 5/10
23/23 [==============================] - 140s 6s/step - loss: 0.6195 - accuracy: 0.6788
Epoch 6/10
23/23 [==============================] - 138s 6s/step - loss: 0.6159 - accuracy: 0.6788
Epoch 7/10
23/23 [==============================] - 160s 7s/step - loss: 0.6131 - accuracy: 0.6776
Epoch 8/10
23/23 [==============================] - 135s 6s/step - loss: 0.6076 - accuracy: 0.6772
Epoch 9/10
23/23 [==============================] - 133s 6s/step - loss: 0.5999 - accuracy: 0.6734
Epoch 10/10
23/23 [==============================] - 136s 6s/step - loss: 0.5520 - accuracy: 0.7053


In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

{'verbose': 1,
 'batch_size': 256,
 'epochs': 10,
 'solver': 'adam',
 'units': 50,
 'build_fn': <function __main__.create_gru_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam')>}

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_units','param_dropout_rate','param_regularizer','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Halophilic/ESM23BGRU2.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

178/178 [==============================] - 102s 569ms/step
acc: 0.7562610229276896
f1: 0.8362559241706162
mcc: 0.39689518769832893
sn: [0.9135899450117831, 0.9156118143459916, 0.9266823775857554, 0.9279958409149986, 0.9205020920502092, 0.9183303085299456, 0.9187353018029788, 0.9149491260109575, 0.9194648478488983, 0.9184936806809388, 0.9146434960388449, 0.9152586430985183, 0.9188210961737332, 0.9182146542827657, 0.9123485434390307, 0.9202518363064008, 0.9157127991675338, 0.91785436641617, 0.9138064516129032, 0.9184360435007768, 0.925589836660617, 0.9115964638585543, 0.9175882043180621, 0.9144805876180483, 0.9171270718232044, 0.9176685833768949, 0.9183303085299456, 0.9139240506329114, 0.9082062072593372, 0.9178154825026511, 0.914778578784758, 0.9169312169312169, 0.9167750325097529, 0.92, 0.916231732776618, 0.9105648535564853, 0.9109677419354839, 0.9155327342747112, 0.9174287954010975, 0.9141981613891726, 0.9163673678809646, 0.9134766649187205, 0.9118500923726577, 0.9147757255936676, 0.9

(0.005616736657918823, 0.013084198147822303, 0.004279407147895261)

In [ ]:
predicted_gru = classifier.predict(X_test)
predicted_gru = np.where(predicted_gru > 0.5, 1, 0)
predicted_gru = np.reshape(predicted_gru,(len(predicted_gru),)).astype(int)
error_rate(Y_test, predicted_gru)

43/43 [==============================] - 23s 535ms/step
acc: 0.81047197640118
f1: 0.8926931106471816
mcc: 0.08770083116577813
sn: [0.9087837837837838, 0.9223549488054608, 0.9180187873612298, 0.9112068965517242, 0.9243697478991597, 0.927038626609442, 0.9038785834738617, 0.92, 0.9072164948453608, 0.9163793103448276, 0.910333048676345, 0.9135180520570949, 0.9044750430292599, 0.9105485232067511, 0.9224211423699915, 0.9146029035012809, 0.9271012006861064, 0.9231433506044905, 0.9225473321858864, 0.9302325581395349, 0.9219677692960135, 0.9080944350758853, 0.9281437125748503, 0.9186241610738255, 0.9144568006843456, 0.9172354948805461, 0.9130801687763713, 0.9222316145393068, 0.9108826049700086, 0.92, 0.918872758326217, 0.9242553191489362, 0.9184188393608074, 0.9042735042735043, 0.9273648648648649, 0.9203463203463204, 0.9204448246364414, 0.9087809036658141, 0.9086918349429324, 0.9121447028423773, 0.9236641221374046, 0.9028374892519346, 0.9146029035012809, 0.9282051282051282, 0.9069767441860465, 

(0.010436783873960619, 0.03260543714441914, 0.006528000159460506)

### LSTM with GridSearch

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from time import time
from tensorflow.keras import layers

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
max_length = 1280

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
def create_lstm_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam'):
    x_input = layers.Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    lstm = LSTM(units, activity_regularizer=l2(regularizer))(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(lstm)
    #lstm2 =LSTM(units, activity_regularizer=l2(dropout_rate))(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    #x2 = Dropout(dropout_rate)(lstm2)
    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=solver,#tf.keras.optimizers.Adam(learning_rate=learning_rate_init),
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
lstm_model = create_lstm_model()
lstm_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 2560, 1)]         0         
                                                                 
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dropout_8 (Dropout)         (None, 50)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_lstm_model, verbose=1, batch_size=256, epochs=5)
# define parameters and values for grid search

parameters = {
    'units':[50], # 1024
    #'learning_rate_init': [0.001, 0.01],
    'solver':['adam'],
    #'epochs':[3,5,10],
    #'dropout_rate':[0.0,0.05, 0.1], #0.05
    #'regularizer':[0.0,0.05, 0.1],
}

<ipython-input-96-7d51c6a25297>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_lstm_model, verbose=1, batch_size=256, epochs=5)


In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

Epoch 1/5
23/23 [==============================] - 72s 3s/step - loss: 0.6609 - accuracy: 0.6781
Epoch 2/5
23/23 [==============================] - 70s 3s/step - loss: 0.6297 - accuracy: 0.6788
Epoch 3/5
23/23 [==============================] - 73s 3s/step - loss: 0.6268 - accuracy: 0.6788
Epoch 4/5
23/23 [==============================] - 94s 4s/step - loss: 0.6259 - accuracy: 0.6788
Epoch 5/5
23/23 [==============================] - 71s 3s/step - loss: 0.6165 - accuracy: 0.6794


In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

{'verbose': 1,
 'batch_size': 256,
 'epochs': 5,
 'solver': 'adam',
 'units': 50,
 'build_fn': <function __main__.create_lstm_model(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam')>}

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_units','param_dropout_rate','param_regularizer','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Halophilic/ESM23BLSTM2.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

178/178 [==============================] - 57s 316ms/step
acc: 0.627689594356261
f1: 0.7643710235517357
mcc: -0.05678453135191831
sn: [0.8908091123330715, 0.8995253164556962, 0.9007593610892904, 0.8921237327787886, 0.8964435146443515, 0.8843660876328753, 0.8881630519989548, 0.8896425776154449, 0.8869359916054564, 0.8885736394119165, 0.8852542806031178, 0.8936833896542761, 0.890382626680455, 0.8833849329205367, 0.8821861304459913, 0.8934942287513117, 0.8920395421436004, 0.8877947654832858, 0.8877419354838709, 0.8979803210771621, 0.8864402385273529, 0.8848153926157046, 0.8957345971563981, 0.8921825813221406, 0.8866087871612733, 0.8834291688447464, 0.8913663469017371, 0.8870886075949367, 0.8958442924776433, 0.8759278897136797, 0.8877445932028837, 0.8896825396825396, 0.893628088426528, 0.8906493506493507, 0.8885699373695198, 0.8909518828451883, 0.8936774193548387, 0.8942233632862644, 0.8881630519989548, 0.8970888661899897, 0.8904566444330426, 0.8985317252228632, 0.8857218263394036, 0.89472

(0.0062491648543549876, 0.012456692575796834, 0.00483836971589913)

In [ ]:
predicted_lstm = classifier.predict(X_test)
predicted_lstm = np.where(predicted_lstm > 0.5, 1, 0)
predicted_lstm = np.reshape(predicted_lstm,(len(predicted_lstm),)).astype(int)
error_rate(Y_test, predicted_lstm)

43/43 [==============================] - 14s 322ms/step
acc: 0.7927728613569321
f1: 0.8820814099874107
mcc: 0.030856399446218957
sn: [0.887668918918919, 0.9027303754266212, 0.8889837745516652, 0.9025862068965518, 0.9058823529411765, 0.8832618025751073, 0.8920741989881956, 0.9004255319148936, 0.9080756013745704, 0.906896551724138, 0.8958155422715628, 0.9059613769941226, 0.9036144578313253, 0.9080168776371308, 0.8985507246376812, 0.8915456874466268, 0.8962264150943396, 0.8981001727115717, 0.9027538726333907, 0.905254091300603, 0.909245122985581, 0.9139966273187183, 0.9093242087254063, 0.9018456375838926, 0.8956372968349017, 0.9027303754266212, 0.9046413502109705, 0.9112426035502958, 0.8928877463581834, 0.8961702127659574, 0.9017933390264731, 0.8927659574468085, 0.8982338099243061, 0.9102564102564102, 0.9138513513513513, 0.9038961038961039, 0.9059024807527801, 0.8934356351236147, 0.9148375768217735, 0.8957795004306632, 0.9016115351993215, 0.8968185726569218, 0.9180187873612298, 0.91794871

(0.010622963617078833, 0.029662912284688325, 0.006730872183436696)

### BiLSTM with GridSearch

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, Input, Embedding, LSTM, Dropout, Dense, InputLayer
from tensorflow.keras.models import Model, Sequential
from keras import regularizers
from keras.layers import Embedding, Bidirectional
from keras.regularizers import l2
from sklearn.metrics import make_scorer
from keras.wrappers.scikit_learn import KerasClassifier
from time import time

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
input_shape = X_train[1,:].shape

In [ ]:
input_shape

(2560, 1)

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
num_words = 22
num_classes = 1
n_cv = 3
num_hiddens = 1280
num_steps = 10
num_layers = 1
max_length = 2560

In [ ]:
def create_blstm_model1(dropout_rate=0.0, units=50, learning_rate_init=0.001, regularizer=0.0, solver='adam'):
    x_input = Input(input_shape)
    #emb = Embedding(21, units, input_length=max_length)(x_input)
    bi_rnn = Bidirectional(LSTM(units, activity_regularizer=l2(dropout_rate),return_sequences=True))(x_input) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x = Dropout(dropout_rate)(bi_rnn)
    bi_rnn2 = Bidirectional(LSTM(units, activity_regularizer=l2(dropout_rate)))(x) #kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)
    x2 = Dropout(dropout_rate)(bi_rnn2)
    x_output = Dense(1, activation='sigmoid')(x2)

    model = Model(inputs=x_input, outputs=x_output)
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_init),#solver,#
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [ ]:
blstm1 = create_blstm_model1()

In [ ]:
blstm1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 2560, 1)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 2560, 100)        20800     
 l)                                                              
                                                                 
 dropout_10 (Dropout)        (None, 2560, 100)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 100)              60400     
 nal)                                                            
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 101 

In [ ]:
# Early Stopping
#es = EarlyStopping(monitor='val_loss', patience=150, verbose=1)

#history = blstm1.fit(train_pad, y_train,
#                        validation_data=(val_pad, y_val),
#                        callbacks=[es],
#                        epochs=3, batch_size=256, verbose=1
#                        )

In [ ]:
#predicted_training_labels = blstm1.predict(train_pad)
#predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
#predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
#error_rate(y_train, predicted_training_labels)

In [ ]:
#predicted_testing_labels = blstm1.predict(val_pad)
#predicted_testing_labels = np.where(predicted_testing_labels > 0.5, 1, 0)
#predicted_testing_labels = np.reshape(predicted_testing_labels,(len(predicted_testing_labels),)).astype(int)
#error_rate(y_val, predicted_testing_labels)

In [ ]:
start = time()

# create model
model = KerasClassifier(build_fn=create_blstm_model1, verbose=1, batch_size=256, epochs=3)
# define parameters and values for grid search

parameters = {
    'units':[50,],
    #'learning_rate_init': [0.001, 0.01],
    #'epochs': [2,3,5,10],
    'solver':['adam'],
    #'dropout_rate':[0.0,0.05, 0.1],
    #'regularizer':[0.0,0.05, 0.1],
}

<ipython-input-120-2f1cd1c262b7>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_blstm_model1, verbose=1, batch_size=256, epochs=3)


In [ ]:
metrics = {'accuracy':make_scorer(accuracy_score,greater_is_better=True),'f1':make_scorer(f1_score,greater_is_better=True),'mcc':make_scorer(matthews_corrcoef,greater_is_better=True)}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=10, scoring=metrics, refit='mcc')
grid_result = grid.fit(X_train, Y_train)

TerminatedWorkerError: ignored

In [ ]:
classifier = grid.best_estimator_

In [ ]:
params = classifier.get_params()
params

In [ ]:
#cv_results = DataFrame(grid.cv_results_)

#cv_results[['param_units','param_activation','param_dropout_rate','param_regularizer','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
df =pd.DataFrame(grid.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/Halophilic/ESM23BBLSTM2.xlsx')

In [ ]:
predicted_training_labels = classifier.predict(X_train)
predicted_training_labels = np.where(predicted_training_labels > 0.5, 1, 0)
predicted_training_labels = np.reshape(predicted_training_labels,(len(predicted_training_labels),)).astype(int)
error_rate(Y_train, predicted_training_labels)

In [ ]:
predicted_blstm = classifier.predict(X_test)
predicted_blstm = np.where(predicted_blstm > 0.5, 1, 0)
predicted_blstm = np.reshape(predicted_blstm,(len(predicted_blstm),)).astype(int)
error_rate(Y_test, predicted_blstm)

In [ ]:
classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, predicted_blstm, labels=classes)
conf_matrix(confusion_matrix_data)
confusion_matrix_data